<a href="https://colab.research.google.com/github/InfDev00/National-Language-Processing/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5%20term%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기계학습 term project
  류희철(2019320136)

#### 1. 실험 내용에 대한 전체 요약
    
  PASS
    


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

#### 2. 데이터 출처 및 선정한 데이터에 대한 설명
    
  제가 선정한 데이터는 Video Game Sales (https://www.kaggle.com/gregorut/videogamesales) 입니다. 10만 부 이상이 팔린 비디오 게임의 리스트이며 vgchartz.com의 scrape에서 만들어졌습니다.

  이 데이터는 Rank, Name, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales의 11개의 feature를 지니고 있으며 약 1만 6천개의 instance를 지니고 있습니다.
  이 중 Name, Platform, Genre, Publisher는 categorical feature이며 나머지는 numerical feature입니다.

#### 3. 실험 설계 및 방법
##### a. 자신이 선정한 모델 및 모델 선정에 대한 이유
  제가 선정한 모델은 AdaBoost, Decision Tree, SVM, MLP, Perceptron입니다.
  AdaBoost가 Decision Tree를 Boosting하여 사용하기에 둘의 성능을 비교하기 위하여 선정하였으며 마찬가지의 이유로 MLP와 Perceptron도 성능 차이를 확인하기 위하여 선정하였습니다. 마지막으로 SVM은 input space를 수정하여 linear_model을 non_linear_model처럼 사용한다는 점이 흥미로워서 선정하였습니다.

b. 데이터 전처리 과정

In [ ]:
url = "https://raw.githubusercontent.com/InfDev00/TestReposit/main/vgsales.csv"
data = pd.read_csv(url)
data.dropna()

def filter(data, col, threshold):
  num_series = data[col].value_counts()
  for etc in num_series.index:
    if(num_series[etc]<threshold):
      index = data.index[data[col]==etc]
      data.loc[index, col] = 'etc'

filter(data, "Publisher", len(data)*0.001)
filter(data, "Platform", len(data)*0.001)

print(data["Publisher"].value_counts())

etc                         1793
Electronic Arts             1351
Activision                   975
Namco Bandai Games           932
Ubisoft                      921
                            ... 
Natsume                       17
SCi                           17
DreamCatcher Interactive      17
O-Games                       17
Rocket Company                17
Name: Publisher, Length: 104, dtype: int64


In [ ]:
def one_hot_encoding(col, data):
  encode_data = pd.get_dummies(data[col], prefix = col)
  return pd.concat([data.drop(col, axis = 1),encode_data],axis=1)

encoded_data = one_hot_encoding(["Platform", "Publisher"], data)
encoded_data.head()

,Rank,Name,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Platform_2600,...,Publisher_Ubisoft,Publisher_Universal Interactive,Publisher_Unknown,Publisher_Virgin Interactive,Publisher_Vivendi Games,Publisher_Warner Bros. Interactive Entertainment,Publisher_Zoo Digital Publishing,Publisher_Zoo Games,Publisher_Zushi Games,Publisher_etc
0,1,Wii Sports,2006.0,Sports,41.49,29.02,3.77,8.46,82.74,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Super Mario Bros.,1985.0,Platform,29.08,3.58,6.81,0.77,40.24,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Mario Kart Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Wii Sports Resort,2009.0,Sports,15.75,11.01,3.28,2.96,33.00,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Pokemon Red/Pokemon Blue,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def Z_score(data):
  mean = data.mean();
  std = data.std()
  return (data-mean) / std

y = encoded_data[["Genre"]]
X = encoded_data.drop(["Name","Genre"], axis = 1)
norm_X = Z_score(X)

drop_na = pd.concat([y, norm_X],axis=1).dropna()

data_y = drop_na[["Genre"]]
data_X = drop_na.drop(["Genre"], axis = 1)

for col in norm_X:
  threshold = 1.0
  norm_X[col] = (norm_X[col] > threshold).astype(float)
 
norm_X["Rank"].value_counts()



0.0    13091
1.0     3507
Name: Rank, dtype: int64

In [ ]:
#def categorical(data, columns, n):
#  for col in columns:
#    labels = range(n)
#    data[col] = pd.cut(data[col], n, labels=labels)


#  return data

#cate_data = categorical(encoded_data, ["Year", "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"], 5)

#y = cate_data["Genre"]
#X = cate_data.drop(["Name", "Genre"], axis=1)

#cate_data.head()

,Rank,Name,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Platform_2600,...,Publisher_Ubisoft,Publisher_Universal Interactive,Publisher_Unknown,Publisher_Virgin Interactive,Publisher_Vivendi Games,Publisher_Warner Bros. Interactive Entertainment,Publisher_Zoo Digital Publishing,Publisher_Zoo Games,Publisher_Zushi Games,Publisher_etc
0,1,Wii Sports,3,Sports,4,4,1,4,4,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Super Mario Bros.,0,Platform,3,0,3,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Mario Kart Wii,3,Racing,1,2,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Wii Sports Resort,3,Sports,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Pokemon Red/Pokemon Blue,1,Role-Playing,1,1,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0


c. 실험 과정

In [ ]:
d_tree_classifier = DecisionTreeClassifier(random_state = 42)
ada_classifier = AdaBoostClassifier(random_state = 42)
per_classifier = Perceptron(random_state = 42)
mlp_classifier = MLPClassifier(random_state = 42)
svm_classifier = SVC(random_state = 42)

k_fold = StratifiedKFold(n_splits = 5)


X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, random_state=42)
d_tree_list = ["Decision_Tree"]
ada_list = ["AdaBoost"]
per_list = ["Perceptron"]
mlp_list = ["MLP"]
svm_list = ["SVM"]

for train_index, val_index in k_fold.split(X_train, y_train):
  X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
  y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]

  d_tree = d_tree_classifier.fit(X_train_cv, y_train_cv) 
  ada = ada_classifier.fit(X_train_cv, y_train_cv)
  per = per_classifier.fit(X_train_cv, y_train_cv)
  mlp = mlp_classifier.fit(X_train_cv, y_train_cv)
  svm = svm_classifier.fit(X_train_cv, y_train_cv)

  d_tree_list.append(d_tree.score(X_val_cv, y_val_cv))
  ada_list.append(ada.score(X_val_cv, y_val_cv))
  per_list.append(per.score(X_val_cv, y_val_cv))
  mlp_list.append(mlp.score(X_val_cv, y_val_cv))
  svm_list.append(svm.score(X_val_cv, y_val_cv))

def get_mean(d_list):
  sum = 0
  n = len(d_list)
  for i in d_list[1:]:
    sum+=i
  return sum/n

max = 0
max_val = ""
for i in [d_tree_list, ada_list, per_list, mlp_list, svm_list]:
  mean = get_mean(i)
  if mean > max:
    max = mean
    max_val = i[0]

print(f"{max_val} : {max}")


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't

KeyboardInterrupt: ignored